<h3 style="text-align: center;"><b>Работа с изображениями.</b></h3>


In [1]:
import matplotlib.pyplot as plt
from pathlib import Path
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader


from sentence_transformers import SentenceTransformer
from PIL import Image


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.neighbors import NearestNeighbors
from catboost import CatBoostClassifier, Pool

# Загрузка данных.

In [2]:
# Путь к папке с датасетом
DATA_DIR = Path("C:/Users/vds/Work/Programming Stuff/ecup/data")

In [3]:
# Загрузка
df_train = pd.read_csv(DATA_DIR / "train.csv", index_col='id')
df_test = pd.read_csv(DATA_DIR / "test_full.csv", index_col='id')

In [4]:
# Импортируем скрипт для работы с данными
import sys
sys.path.append(str(Path.cwd().parent))
from scripts import data_preprocess

In [5]:
import warnings
warnings.filterwarnings('ignore')

# Обрабатываем и получаем данные
df_train_num, df_train_text = data_preprocess.clean_data(df_train)
df_test_num, df_test_text = data_preprocess.clean_data(df_test, type='test')

# Работа с изображениями.

In [6]:
# Путь к папке с картинками
DATA_DIR = Path("C:/Users/vds/Work/Programming Stuff/ecup/data/images")
TRAIN_DIR = Path(DATA_DIR / "train")
TEST_DIR = Path(DATA_DIR / "test")

## 1) Создание эмбеддингов изображения

In [7]:
df_train[df_train['ItemID'] == 3490]

,resolution,brand_name,description,name_rus,CommercialTypeName4,rating_1_count,rating_2_count,rating_3_count,rating_4_count,rating_5_count,...,ExemplarReturnedCountTotal30,ExemplarReturnedCountTotal90,ExemplarReturnedValueTotal7,ExemplarReturnedValueTotal30,ExemplarReturnedValueTotal90,ItemVarietyCount,ItemAvailableCount,seller_time_alive,ItemID,SellerID
id,,,,,,,,,,,,,,,,,,,,,
6863,0,Levsha kaluga,<p>Мы используем только НОВЫЕ комплектующие пр...,Levsha kaluga Системный блок Игровой компьютер...,Настольный компьютер,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,366.0,366.0,117.0,3490,442


In [8]:
from sentence_transformers import SentenceTransformer
from PIL import Image

# Загружаем предобученный CLIP
model = SentenceTransformer("clip-ViT-B-32", device="cuda")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Функция: получение эмбеддингов картинок

In [9]:
def compute_image_embeddings(df, img_dir, batch_size=32):
    embeddings = []
    ids = []

    img_paths = [img_dir / f"{img_id}.png" for img_id in df["ItemID"]]

    for i in tqdm(range(0, len(img_paths), batch_size)):
        batch_paths = img_paths[i : i + batch_size]
        batch_imgs = []

        for path in batch_paths:
            try:
                img = Image.open(path).convert("RGB")
            except Exception as e:
                print(f"Ошибка при открытии {path}: {e}")
                # если картинки нет или битая, добавим "пустышку"
                img = Image.new("RGB", (224, 224), (255, 255, 255))
            batch_imgs.append(img)

        # получаем эмбеддинги
        batch_emb = model.encode(batch_imgs, batch_size=batch_size, convert_to_numpy=True, show_progress_bar=False)
        embeddings.append(batch_emb)

        ids.extend(df.index[i : i + batch_size])

    embeddings = np.vstack(embeddings)

    return pd.DataFrame(embeddings, index=ids)

Генерация эмбеддингов

In [ ]:
train_img_emb = compute_image_embeddings(df_train, TRAIN_DIR, batch_size=32)

In [10]:
test_img_emb = compute_image_embeddings(df_test, TEST_DIR, batch_size=32)

  5%|▌         | 53/981 [01:11<22:59,  1.49s/it]

Ошибка при открытии C:\Users\vds\Work\Programming Stuff\ecup\data\images\test\185232.png: [Errno 2] No such file or directory: 'C:\\Users\\vds\\Work\\Programming Stuff\\ecup\\data\\images\\test\\185232.png'


 10%|▉         | 94/981 [02:13<20:15,  1.37s/it]

Ошибка при открытии C:\Users\vds\Work\Programming Stuff\ecup\data\images\test\167026.png: [Errno 2] No such file or directory: 'C:\\Users\\vds\\Work\\Programming Stuff\\ecup\\data\\images\\test\\167026.png'


 12%|█▏        | 115/981 [02:45<20:12,  1.40s/it]

...

 66%|██████▋   | 650/981 [15:25<07:16,  1.32s/it]

Ошибка при открытии C:\Users\vds\Work\Programming Stuff\ecup\data\images\test\128913.png: [Errno 2] No such file or directory: 'C:\\Users\\vds\\Work\\Programming Stuff\\ecup\\data\\images\\test\\128913.png'


 71%|███████   | 692/981 [16:27<06:39,  1.38s/it]

Ошибка при открытии C:\Users\vds\Work\Programming Stuff\ecup\data\images\test\7415.png: [Errno 2] No such file or directory: 'C:\\Users\\vds\\Work\\Programming Stuff\\ecup\\data\\images\\test\\7415.png'


100%|██████████| 981/981 [22:47<00:00,  1.39s/it]


Сохранение

In [ ]:
train_img_emb.to_parquet(DATA_DIR / "train_clip_img_emb.parquet")

In [11]:
test_img_emb.to_parquet(DATA_DIR / "test_clip_img_emb.parquet")

Загрузка

In [8]:
train_img_emb = pd.read_parquet(DATA_DIR / "train_clip_img_emb.parquet")
test_img_emb = pd.read_parquet(DATA_DIR / "test_clip_img_emb.parquet")

## 2) Считывание текста с изображения

In [10]:
import easyocr

# Инициализация OCR (английский + русский)
ocr = easyocr.Reader(['en', 'ru'], gpu=True)

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.1% Complete

In [11]:
def extract_texts(df, img_dir):
    texts, has_text = [], []
    nan_count = 0
    
    for img_id in tqdm(df["ItemID"]):
        path = img_dir / f"{img_id}.png"
        try:
            result = ocr.readtext(str(path), detail=1)  # detail=1 → вернёт bbox, текст и уверенность
            if result:
                # Собираем все найденные строки
                text_parts = [line[1] for line in result]  
                text = " ".join(text_parts)
                texts.append(text)
                has_text.append(1)
            else:
                texts.append("")
                has_text.append(0)
        except Exception as e:
            nan_count += 1
            texts.append("")
            has_text.append(0)

    print(f"Преобразование выполнено, нет изображений у {nan_count} товаров из {len(df)} ({nan_count/len(df):.2%})")
    return texts, has_text

Применение

In [ ]:
df_test["ocr_text"], df_test["has_text"] = extract_texts(df_test, TEST_DIR)
df_train["ocr_text"], df_train["has_text"] = extract_texts(df_train, TRAIN_DIR)

  0%|          | 655/197198 [09:54<49:32:01,  1.10it/s] 


KeyboardInterrupt: 

In [ ]:
# Сохраним обновлённые таблицы
os.makedirs("ocr", exist_ok=True)

df_train.to_csv("ocr/train_with_ocr.csv", index=True)
df_test.to_csv("ocr/test_with_ocr.csv", index=True)

print("OCR завершён, данные сохранены.")

# Обучение модели.

### 1) Уменьшим размерность векторов.

In [73]:
from sklearn.decomposition import PCA

pca_vectors = PCA(n_components=142, random_state=34)

train_img_emb_r = pca_vectors.fit_transform(train_img_emb)

### 2) Получим векторы текста.

In [74]:
embeddings, embeddings_test = data_preprocess.load_text_embeddings()

### 3) Подготовим данные для модели.

In [75]:
# Возьмем числовные данные
num_data = df_train_num.drop(columns=['resolution']).values

# Также возьмем категориальные признаки
cat_cols = ["brand_name", "CommercialTypeName4"]
cat_data = df_train_text[cat_cols].astype(str)

# Теперь объединяем: эмбеддинги + изображения + числовые
X_num = np.concatenate([embeddings, train_img_emb_r, num_data], axis=1)  # (N, D + num_features)

# Целевая переменная
y = df_train_num["resolution"].astype(int).values

In [76]:
from sklearn.model_selection import train_test_split

# Трейн/тест сплит
X_train_num, X_val_num, y_train, y_val, train_cat, val_cat = train_test_split(
    X_num, y, cat_data, test_size=0.2, stratify=y, random_state=42
)

In [77]:
# --- Объединяем все в один датафрейм
X_train = pd.concat(
    [pd.DataFrame(X_train_num), train_cat.reset_index(drop=True)], axis=1
)
X_val = pd.concat(
    [pd.DataFrame(X_val_num), val_cat.reset_index(drop=True)], axis=1
)

In [78]:
# --- Создаем пулы для catboost

# Категориальные признаки теперь — последние len(cat_cols) колонок
cat_features_idx = list(range(X_train_num.shape[1], X_train_num.shape[1] + len(cat_cols)))

train_pool = Pool(X_train, label=y_train, cat_features=cat_features_idx)
val_pool = Pool(X_val, label=y_val, cat_features=cat_features_idx)

### 4) Обучение

In [1]:
# Модель
model = CatBoostClassifier(
    iterations=500,
    depth=11,
    learning_rate=0.05,
    eval_metric="F1",
    random_seed=42,
    od_type="Iter", 
    od_wait=50,
    task_type="GPU" if torch.cuda.is_available() else "CPU"
)

model.fit(train_pool, eval_set=val_pool,
    verbose=100,
    use_best_model=True,      # Использовать лучшую модель по валидации
    plot=True                 # Построить график обучения
)

NameError: name 'CatBoostClassifier' is not defined

In [80]:
# Предсказания
y_pred = model.predict(val_pool)
print("\n=== Classification report ===")
print(classification_report(y_val, y_pred, digits=4))


=== Classification report ===
              precision    recall  f1-score   support

           0     0.9836    0.9904    0.9870     36830
           1     0.8505    0.7674    0.8068      2610

    accuracy                         0.9757     39440
   macro avg     0.9171    0.8789    0.8969     39440
weighted avg     0.9748    0.9757    0.9751     39440



### 5) Получение ответов на тесте.

Подготовка данных

In [59]:
test_img_emb_r = pca_vectors.transform(test_img_emb)

In [60]:
# Возьмем числовные данные
num_data_test = df_test_num.values

# Также возьмем категориальные признаки
cat_cols = ["brand_name", "CommercialTypeName4"]
cat_data_test = df_test_text[cat_cols].astype(str)

# Теперь объединяем: эмбеддинги + числовые
X_test_num = np.concatenate([embeddings_test, test_img_emb_r, num_data_test], axis=1)  # (N, D + num_features)

In [61]:
# Объединяем в датафрейм для CatBoost
X_test = pd.concat([pd.DataFrame(X_test_num), cat_data_test.reset_index(drop=True)], axis=1)

test_pool = Pool(X_test, cat_features=cat_features_idx)  # cat_features_idx те же, что для валидации

y_pred_test = model.predict(test_pool)

Предсказание

In [62]:
test_predictions = model.predict(test_pool)

submission = pd.DataFrame({
    'id': df_test.index,
    'prediction': test_predictions
})

submission.to_csv('submission.csv', index=False)


print(f"Создан файл submission.csv с {len(submission)} предсказаниями")
print(f"Распределение предсказаний:")
print(submission['prediction'].value_counts())
print()

Создан файл submission.csv с 22760 предсказаниями
Распределение предсказаний:
prediction
0    21442
1     1318
Name: count, dtype: int64

